In [200]:
#prepare
import numpy as np
workers =12
shifts = 3
# Create a 3x21 matrix with at least one value that is not 3 in each column
requests = np.random.randint(low=1, high=4, size=(workers, shifts*7))

for i in range(shifts*7):
    # If all the values in the current column are 3, assign a value of 1 or 2 to a random row
    if np.all(requests[:, i] == 3):
        requests[np.random.randint(0, workers), i] = np.random.randint(1, 3)



In [202]:
import random

parents = []
childs = []

#create random valid state
def Create_random_state():
  open = list(range(0,workers))
  after_night = 0 
  # state = np.random.randint(0, workers, size=shifts*7)
  # לשים לב עשיתי מודולו 3 ממחשבה שיש 3 משמרות אם יש יותר צריך לחשוב מה החוקים במקרה כזה
  state = []
  for i in range(shifts*7):
    if (i%3 == 0 and i != 0):
      open = list(range(0,workers))
      open.remove(after_night)
    if(i%3 ==1 and i!= 1):
      open.append(after_night)

    rand_worker = random.choice(open)
    state.append(rand_worker)
    open.remove(rand_worker)
    if(i%3 ==2):
      after_night = rand_worker
  return state





#chack if two states are the same
def Equal(state1 , state2):
  for i in range(shifts*7):
      if (state1[i] != state2[i]):
        return False
  return True

#check if state list containts a state
def check_contains_state(state , state_arr):
  for s in state_arr:
    if (Equal(s,state)):
      return True
  return False
  
a = Create_random_state()
print (a)


[9, 4, 1, 9, 6, 1, 6, 10, 7, 3, 2, 4, 1, 8, 2, 7, 1, 6, 0, 7, 8]


In [203]:
#init 
#initialize list of 100 perent states
def Init():
  while (len(parents) <25000):
    new_state = Create_random_state()
    #if not check_contains_state(new_state, parents):
    parents.append(new_state)


Init()


In [204]:
try_state = parents[0]
print (try_state)
#delete later

[11, 0, 8, 7, 10, 6, 0, 6, 2, 0, 7, 3, 4, 8, 10, 9, 5, 8, 0, 6, 9]


In [205]:
# check if state is valid to constaaints
def check_constraint(state):
  for i in range(2,shifts*7-1,shifts):
     if (state[i] == state[i+1]):
       #print('morning night at ',i)
       return False
  for i in range (0,shifts*7,shifts):
    if ((state[i]== state[i+1]) or (state[i]==state[i+2]) or (state[i+1] == state[i+2]) ):
      #print('more then 1 shift\day at',i)
      return False
  return True


def fitness(state):
  not_valid = 123456789
  sum = 0
  if (not check_constraint(state)):
    return not_valid
  for i in range(len(state)):
    sum += requests[state[i]][i]
  score = sum
  return score


In [206]:
#delete later
print(try_state)

cc = check_constraint(try_state)
print(cc)

f = fitness(try_state)
print(f)

[11, 0, 8, 7, 10, 6, 0, 6, 2, 0, 7, 3, 4, 8, 10, 9, 5, 8, 0, 6, 9]
True
45


In [207]:
#return 50% fittest states
def select_half_fittest(states):
  sorted_states = sorted(states, key=fitness)
  half = len(states)//2
  if (half %2 !=0):
    half = half-1
  fittest = sorted_states[:half]
  return fittest

#returns the fitness value of the fitest
def fittest_in_group(states):
  sorted = sorted(states, key=fitness)
  return sorted[0]


In [208]:
#create new generation from two parents
def make_new_gen(state1,state2):
  child = []
  for i in range(shifts*7):
    if (requests[state1[i]][i] < requests[state2[i]][i]):
      child.append(state1[i])
    else:
      child.append(state2[i])
  return child


In [210]:
def schedule(states):
  better = True
  gen0 = select_half_fittest(states)
  print('Starting with', len(gen0), 'states')
  
  while better:
    gen1 = []
    better_counter = 0
    while len(gen0) > 4:
      parent1 = random.choice(gen0)
      gen0.remove(parent1)
      parent2 = random.choice(gen0)
      gen0.remove(parent2)
      child = make_new_gen(parent1, parent2)
      gen1.append(child)
      if fitness(child) < min(fitness(parent1), fitness(parent2)):
        better_counter += 1
    if (len(gen1)<=10):    
      return gen1
    gen0 = select_half_fittest(gen1)  

    print('Generated', len(gen0), 'new states')
  
    if better_counter == 0 or len(gen0) == 1:
      better = False
      
  return gen0








In [211]:
print (len(parents))

25000


In [212]:

child = schedule(parents)
print()

Starting with 12500 states
Generated 3124 new states
Generated 780 new states
Generated 194 new states
Generated 46 new states
Generated 10 new states



In [213]:
for state in child:
  print (state , fitness(state) , '\n')

print(len(child))

[3, 11, 6, 11, 7, 2, 4, 5, 6, 2, 6, 4, 3, 0, 7, 8, 2, 10, 8, 0, 3] 21 

[2, 11, 4, 7, 10, 4, 0, 5, 6, 10, 9, 0, 5, 0, 7, 6, 8, 10, 8, 9, 3] 21 

[11, 7, 5, 7, 5, 2, 0, 5, 9, 10, 9, 4, 6, 0, 7, 6, 2, 10, 7, 0, 1] 21 

3


In [214]:
sorted_childs = sorted(child, key=fitness)
best_schedual = sorted_childs[0]
print(best_schedual , fitness(best_schedual))

[3, 11, 6, 11, 7, 2, 4, 5, 6, 2, 6, 4, 3, 0, 7, 8, 2, 10, 8, 0, 3] 21


In [215]:
sorted_childs

[[3, 11, 6, 11, 7, 2, 4, 5, 6, 2, 6, 4, 3, 0, 7, 8, 2, 10, 8, 0, 3],
 [2, 11, 4, 7, 10, 4, 0, 5, 6, 10, 9, 0, 5, 0, 7, 6, 8, 10, 8, 9, 3],
 [11, 7, 5, 7, 5, 2, 0, 5, 9, 10, 9, 4, 6, 0, 7, 6, 2, 10, 7, 0, 1]]

In [216]:
# visualize 
import pandas as pd
days = [ "Sunday","Monday", "Tuesday", "Wednesday", "Thursday", "Friday", "Saturday"]
shift_names = ["morning" , "evening" , "night"]
day_shift = [d+' '+s for d in days  for s in shift_names ]
vis = pd.DataFrame(best_schedual).T
vis.columns = day_shift
vis = vis.rename(index={0: "worker"})
print('scheduale fitness: ' , fitness(best_schedual) ,"/ optimal: ", shifts*7  )
print("\n")
vis

scheduale fitness:  21 / optimal:  21




,Sunday morning,Sunday evening,Sunday night,Monday morning,Monday evening,Monday night,Tuesday morning,Tuesday evening,Tuesday night,Wednesday morning,...,Wednesday night,Thursday morning,Thursday evening,Thursday night,Friday morning,Friday evening,Friday night,Saturday morning,Saturday evening,Saturday night
worker,3,11,6,11,7,2,4,5,6,2,...,4,3,0,7,8,2,10,8,0,3


In [217]:
import os

def read_text_files(folder_path):
    files = os.listdir(folder_path)
    Allrequests = []

    for file_name in files:
        file_path = os.path.join(folder_path, file_name)
        with open(file_path, 'r') as file:
            content = file.read()
            Allrequests.append(eval(content))

    return Allrequests


In [218]:
a = read_text_files(r"C:\Users\rogov\Desktop\matan easy 4")


In [219]:
Init()
tmp = parents

In [220]:
i=0
for req in a:
    requests = req
    print (requests)
    parents = tmp
    childs = []
    
    child = schedule(parents)
    sorted_childs = sorted(child, key=fitness)
    best_schedual = sorted_childs[0]
    print(fitness(best_schedual))
    with open(r"C:\Users\rogov\Desktop\easy 4 res\easy_4_res.txt", 'a') as file:
         file.write(str(fitness(best_schedual))+'\n')
    

[[3, 3, 3, 3, 2, 3, 3, 2, 3, 3, 3, 2, 3, 3, 3, 1, 3, 1, 2, 3, 3], [3, 3, 3, 3, 1, 1, 3, 3, 3, 3, 2, 1, 3, 3, 3, 3, 3, 3, 3, 2, 1], [2, 1, 3, 3, 3, 2, 3, 1, 3, 3, 2, 2, 3, 1, 2, 2, 3, 2, 3, 3, 3], [3, 3, 2, 2, 3, 2, 1, 3, 3, 3, 3, 3, 3, 2, 3, 3, 3, 3, 3, 3, 3], [3, 2, 1, 3, 3, 3, 3, 3, 3, 2, 1, 3, 2, 3, 2, 2, 3, 1, 1, 3, 3], [3, 3, 3, 3, 2, 3, 3, 3, 3, 3, 3, 3, 3, 2, 3, 2, 3, 3, 3, 3, 2], [3, 3, 3, 3, 2, 3, 2, 2, 2, 3, 3, 3, 3, 3, 3, 3, 2, 3, 3, 3, 3], [1, 3, 2, 3, 3, 3, 3, 1, 3, 3, 3, 3, 3, 1, 2, 2, 1, 3, 2, 3, 3], [3, 1, 1, 3, 3, 3, 1, 3, 2, 3, 3, 1, 3, 3, 3, 2, 2, 3, 1, 3, 2], [3, 3, 3, 3, 3, 3, 2, 3, 2, 3, 3, 3, 3, 3, 3, 2, 3, 1, 1, 3, 3], [2, 3, 3, 3, 3, 2, 3, 3, 3, 3, 2, 3, 3, 3, 2, 3, 2, 2, 1, 3, 3], [1, 2, 1, 3, 3, 3, 2, 1, 3, 3, 3, 3, 3, 3, 2, 3, 2, 3, 2, 3, 3]]
Starting with 12500 states
Generated 3124 new states
Generated 780 new states
Generated 194 new states
Generated 46 new states
123456789
[[3, 1, 2, 3, 3, 2, 3, 3, 3, 3, 2, 3, 3, 3, 2, 3, 3, 3, 2, 3, 2], [3, 1, 3, 2, 1, 

In [221]:
def calculate_average(file_path):
    numbers = []
    count_2147483647 = 0

    # Read numbers from the file
    with open(file_path, 'r') as file:
        for line in file:
            number = int(line.strip())
            if number == 123456789:
                count_2147483647 += 1
            else:
                numbers.append(number)

    # Calculate the average
    average = sum(numbers) / len(numbers)

    # Print the average and count of occurrences of 2147483647
    print(f"Average: {average}")
    print(f"Count of 2147483647: {count_2147483647}")


# Specify the path to your input file
file_path = '/content/output.txt'

# Call the function with the file path
calculate_average(r"C:\Users\rogov\Desktop\easy 4 res\easy_4_res.txt")

Average: 27.516129032258064
Count of 2147483647: 19
